In [1]:
# Need this for this demo to even be possible,
# turns out that jupyter already runs in its own
#  event loop which is NOT pretty to deal with
import nest_asyncio
nest_asyncio.apply()
import asyncio

%load_ext memory_profiler

In [2]:
import requests

print(requests.get("http://0.0.0.0:8080").text)
input()
print(requests.get("http://0.0.0.0:8080/items").text[:1000])
input()
print(requests.get("http://0.0.0.0:8080/items/1").text)

Hello, world

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 

In [3]:
for i in range(650):
    print(i, requests.get("http://0.0.0.0:8080/items/1").text)

0 {"email": "martinlorraine@example.net"}
1 {"email": "martinlorraine@example.net"}
2 {"email": "martinlorraine@example.net"}
3 {"email": "martinlorraine@example.net"}
4 {"email": "martinlorraine@example.net"}
5 {"email": "martinlorraine@example.net"}
6 {"email": "martinlorraine@example.net"}
7 {"email": "martinlorraine@example.net"}
8 {"email": "martinlorraine@example.net"}
9 {"email": "martinlorraine@example.net"}
10 {"email": "martinlorraine@example.net"}
11 {"email": "martinlorraine@example.net"}
12 {"email": "martinlorraine@example.net"}
13 {"email": "martinlorraine@example.net"}
14 {"email": "martinlorraine@example.net"}
15 {"email": "martinlorraine@example.net"}
16 {"email": "martinlorraine@example.net"}
17 {"email": "martinlorraine@example.net"}
18 {"email": "martinlorraine@example.net"}
19 {"email": "martinlorraine@example.net"}
20 {"email": "martinlorraine@example.net"}
21 {"email": "martinlorraine@example.net"}
22 {"email": "martinlorraine@example.net"}
23 {"email": "martinl

In [4]:
from aiohttp import request
import asyncio

async def req(i):
    async with request("get", f"http://0.0.0.0:8080/items/{i}") as r:
        print(await r.content.read())

asyncio.run(req(1))

b'{"email": "martinlorraine@example.net"}'


In [5]:
from throttler import Throttler

async def req(i, t: Throttler):
    print(i, "starting")
    async with t:
        print(i, "inside throttler")
        async with request("get", f"http://0.0.0.0:8080/items/{i}") as r:
            print(i, "before request")
            resp = await r.content.read()
            print(i, "after request")
            return resp

t = Throttler(rate_limit=60, period=10.0)
results = asyncio.run(asyncio.gather(*[asyncio.create_task(req(i, t)) for i in range(70)]))

0 starting
0 inside throttler
1 starting
1 inside throttler
2 starting
2 inside throttler
3 starting
3 inside throttler
4 starting
4 inside throttler
5 starting
5 inside throttler
6 starting
6 inside throttler
7 starting
7 inside throttler
8 starting
8 inside throttler
9 starting
9 inside throttler
10 starting
10 inside throttler
11 starting
11 inside throttler
12 starting
12 inside throttler
13 starting
13 inside throttler
14 starting
14 inside throttler
15 starting
15 inside throttler
16 starting
16 inside throttler
17 starting
17 inside throttler
18 starting
18 inside throttler
19 starting
19 inside throttler
20 starting
20 inside throttler
21 starting
21 inside throttler
22 starting
22 inside throttler
23 starting
23 inside throttler
24 starting
24 inside throttler
25 starting
25 inside throttler
26 starting
26 inside throttler
27 starting
27 inside throttler
28 starting
28 inside throttler
29 starting
29 inside throttler
30 starting
30 inside throttler
31 starting
31 inside thrott

In [6]:
from tenacity import retry, stop_after_attempt, before_log

from throttler import Throttler

import logging

logging.basicConfig(stream=sys.stderr, level=logging.DEBUG)

logger = logging.getLogger(__name__)

# @retry(stop=stop_after_attempt(3), before=before_log(logger, logging.DEBUG))
@retry(stop=stop_after_attempt(3))
async def req(i, t: Throttler):
    async with t:
        async with request("get", f"http://0.0.0.0:8080/items/{i}") as r:
            resp = await r.content.read()
            r.raise_for_status()
            return resp

def run_reqs(rate_limit: int, period: float, n_reqs: int):
    t = Throttler(rate_limit=rate_limit, period=period)
    asyncio.run(asyncio.gather(*[asyncio.create_task(req(i, t)) for i in range(n_reqs)]))

def run_reqs_2(rate_limit: int, period: float, n_reqs: int):
    t = Throttler(rate_limit=rate_limit, period=period)
    tasks = [req(i, t) for i in range(n_reqs)]

async def testr(n):
    return n

def create_tasks(n):
    tasks = [asyncio.create_task(testr(i)) for i in range(n)]

def create_tasks_2(n):
    tasks = [testr(i) for i in range(n)]

In [7]:
# %memit run_reqs(10, 1.0, 1000)
%memit run_reqs_2(10, 1.0, 1000)
# %memit run_reqs_3(10, 1.0, 1000)
%memit create_tasks(1000)
%memit create_tasks(5000)
%memit create_tasks(20_000)
%memit run_reqs_2(10, 1.0, 20_000)

<string>:1: RuntimeWarning: coroutine 'req' was never awaited


peak memory: 66.76 MiB, increment: 0.26 MiB
peak memory: 69.33 MiB, increment: 2.57 MiB
peak memory: 81.25 MiB, increment: 11.88 MiB
peak memory: 106.39 MiB, increment: 25.02 MiB
peak memory: 120.70 MiB, increment: 14.26 MiB


In [26]:
import asyncio
import time

class ThrottledQueue(asyncio.Queue):
    "subclass asyncio.Queue i.e. import all behaviour"

    def __init__(self, per_second, debug=False, maxsize=0, *, loop=None, i=0):
        "Set up some extra vars and then call the original init"

        self.lock = asyncio.Lock()
        self.i = i
        self.per_second = per_second
        self.last_get = time.time() # this is the fastest way... I think?
        self.debug = debug
        super(ThrottledQueue, self).__init__(maxsize=maxsize, loop=loop)

    async def notify(self):
        """
        Signals to the queue that an item is being retried, 
        so pause any get()s by aquiring the lock and throttling before releasing
        """
        async with self.lock:
            await self._throttle()

    async def lock(self, n_seconds: int):
        async with self.lock:
            await asyncio.sleep(n_seconds)

    async def get(self):
        async with self.lock:
            await self._throttle()
            result = await super(ThrottledQueue, self).get()

            self.last_get = time.time()
            return result

    async def retry(self):
        async with self.lock:
            await self._throttle()

    async def _throttle(self):
        elapsed = time.time() - self.last_get
        sleep_time = (1/float(self.per_second)) - elapsed
        if self.debug:
            print(self.i, '- times', f'{elapsed:.5f}', '+', f'{sleep_time:.5f}', '=', self.per_second, '- sizes', self.qsize(), f'{self.qsize() / max(1, self.maxsize):.5f}')
        await asyncio.sleep(max(0, sleep_time)) # Make sure we wait at least 0 seconds

In [27]:
import json

from collections import Counter

import sys
from aiohttp import ClientResponseError

class Sentinel: pass

async def get_all_items(q, cntr: Counter):
    async with request("get", "http://0.0.0.0:8080/items") as r:
        resp = await r.read()
        r.raise_for_status()
        cntr["success"] += 1
        for i, d in enumerate(json.loads(resp)):
            await q.put((i, d))
        await q.put((i, Sentinel))


async def handle_error(e, q):
    print(f"HANDLING ERROR: {e}")
    if e.status == 429:
        q.retry()
    elif e.status == 503:
        q.lock(10)

async def item_worker(q, idx, cntr: Counter, ostream = sys.stdout):
    retrying = False
    while True:
        if not retrying:
            i, d = await q.get()
        if d == Sentinel:
            await q.put((i, Sentinel))
            print(f"worker {idx} exiting")
            return
        try:
            # TODO: actually retry lol, don't just pop a fresh item
            async with request("get", f"http://0.0.0.0:8080/items/{d}") as req:
                resp = await req.read()
                req.raise_for_status()
                print(f"worker {idx} response for #{i}: {resp}", file=ostream)
                cntr["success"] += 1
        except ClientResponseError as e:
            await handle_error(e, q)

def gen_req(idx): pass

def run(per_second, n_workers=10, ostream=sys.stdout, debug=False):
    cntr = Counter()
    q = ThrottledQueue(per_second=per_second, maxsize=1000, debug=debug)
    asyncio.run(
        asyncio.gather(
            asyncio.create_task(get_all_items(q, cntr)),
            *[asyncio.create_task(item_worker(q, i, cntr, ostream)) for i in range(n_workers)],
        )
    )
    return cntr

In [28]:
run(100, n_workers=10)

worker 0 response for #0: b'{"email": "floreschristine@example.net"}'
worker 1 response for #1: b'{"email": "martinlorraine@example.net"}'
worker 2 response for #2: b'{"email": "urosario@example.org"}'
worker 3 response for #3: b'{"email": "bowerslori@example.com"}'
worker 4 response for #4: b'{"email": "kjones@example.org"}'
worker 5 response for #5: b'{"email": "robertreeves@example.org"}'
worker 6 response for #6: b'{"email": "daniel01@example.com"}'
worker 7 response for #7: b'{"email": "alexanderbennett@example.com"}'
worker 8 response for #8: b'{"email": "erikaparsons@example.com"}'
worker 9 response for #9: b'{"email": "christopherramos@example.org"}'
worker 0 response for #10: b'{"email": "umarsh@example.com"}'
worker 1 response for #11: b'{"email": "john42@example.com"}'
worker 2 response for #12: b'{"email": "catherine86@example.org"}'
worker 3 response for #13: b'{"email": "wolfetaylor@example.org"}'
worker 4 response for #14: b'{"email": "julia04@example.com"}'
worker 5 res

Counter({'success': 1001})

In [29]:
with open("results.txt", "w") as ostream:
    run(per_second=100, n_workers=100, ostream=ostream, debug=True)

0 - times 0.00413 + 0.00587 = 100 - sizes 0 0.00000
0 - times 0.00292 + 0.00708 = 100 - sizes 1000 1.00000
0 - times 0.00375 + 0.00625 = 100 - sizes 999 0.99900
0 - times 0.00179 + 0.00821 = 100 - sizes 998 0.99800
0 - times 0.00099 + 0.00901 = 100 - sizes 997 0.99700
0 - times 0.00264 + 0.00736 = 100 - sizes 996 0.99600
0 - times 0.00187 + 0.00813 = 100 - sizes 995 0.99500
0 - times 0.00146 + 0.00854 = 100 - sizes 994 0.99400
0 - times 0.00121 + 0.00879 = 100 - sizes 993 0.99300
0 - times 0.00067 + 0.00933 = 100 - sizes 992 0.99200
0 - times 0.00079 + 0.00921 = 100 - sizes 991 0.99100
0 - times 0.00126 + 0.00874 = 100 - sizes 990 0.99000
0 - times 0.00058 + 0.00942 = 100 - sizes 989 0.98900
0 - times 0.00081 + 0.00919 = 100 - sizes 988 0.98800
0 - times 0.00086 + 0.00914 = 100 - sizes 987 0.98700
0 - times 0.00111 + 0.00889 = 100 - sizes 986 0.98600
0 - times 0.00132 + 0.00868 = 100 - sizes 985 0.98500
0 - times 0.00115 + 0.00885 = 100 - sizes 984 0.98400
0 - times 0.00216 + 0.00784 =

In [30]:
run(per_second=100, n_workers=10, debug=True)

0 - times 0.00380 + 0.00620 = 100 - sizes 0 0.00000
0 - times 0.00152 + 0.00848 = 100 - sizes 1000 1.00000
0 - times 0.00142 + 0.00858 = 100 - sizes 999 0.99900
worker 0 response for #0: b'{"email": "floreschristine@example.net"}'
worker 1 response for #1: b'{"email": "martinlorraine@example.net"}'
0 - times 0.00125 + 0.00875 = 100 - sizes 998 0.99800
worker 2 response for #2: b'{"email": "urosario@example.org"}'
0 - times 0.00184 + 0.00816 = 100 - sizes 997 0.99700
worker 3 response for #3: b'{"email": "bowerslori@example.com"}'
0 - times 0.00092 + 0.00908 = 100 - sizes 996 0.99600
worker 4 response for #4: b'{"email": "kjones@example.org"}'
0 - times 0.00107 + 0.00893 = 100 - sizes 995 0.99500
worker 5 response for #5: b'{"email": "robertreeves@example.org"}'
0 - times 0.00076 + 0.00924 = 100 - sizes 994 0.99400
worker 6 response for #6: b'{"email": "daniel01@example.com"}'
0 - times 0.00189 + 0.00811 = 100 - sizes 993 0.99300
worker 7 response for #7: b'{"email": "alexanderbennett@e

Counter({'success': 1001})

## Fill the Queue beforehand

In [31]:
import json

from collections import Counter

import sys
from aiohttp import ClientResponseError

class Sentinel: pass

async def get_all_items(q, cntr: Counter):
    async with request("get", "http://0.0.0.0:8080/items") as r:
        resp = await r.read()
        r.raise_for_status()
        cntr["success"] += 1
        for i, d in enumerate(json.loads(resp)):
            await q.put((i, d))
        await q.put((i, Sentinel))


async def handle_error(e, q):
    print(f"HANDLING ERROR: {e}")
    if e.status == 429:
        q.retry()
    elif e.status == 503:
        q.lock(10)

async def item_worker(q, idx, cntr: Counter, ostream = sys.stdout):
    while True:
        i, d = await q.get()
        if d == Sentinel:
            await q.put((i, Sentinel))
            print(f"worker {idx} exiting")
            return
        try:
            async with request("get", f"http://0.0.0.0:8080/items/{d}") as req:
                resp = await req.read()
                req.raise_for_status()
                print(f"worker {idx} response for #{i}: {resp}", file=ostream)
                cntr["success"] += 1
        except ClientResponseError as e:
            await handle_error(e, q)

def run(per_second, n_workers=10, ostream=sys.stdout, debug=False):
    cntr = Counter()
    q = ThrottledQueue(per_second=per_second, maxsize=1000, debug=debug)
    asyncio.run(
        asyncio.gather(
            asyncio.create_task(get_all_items(q, cntr)),
            *[asyncio.create_task(item_worker(q, i, cntr, ostream)) for i in range(n_workers)],
        )
    )
    return cntr

In [32]:
import asyncio
from itertools import count

async def _fill_queue(q, items):
    for idx, i in enumerate(items):
        await q.put((idx, i))
    await q.put((idx, Sentinel))
    return q

async def _unpack_queue(q):
    l = list()
    for idx in count():
        i, d = await q.get()
        print(d)
        if d == Sentinel:
            print("exiting")
            return l
        l.append(d)
    return l

def unpack_queue(q):
    return asyncio.run(asyncio.create_task(_unpack_queue(q)))


In [33]:
q = asyncio.run(_fill_queue(asyncio.Queue(), list(range(1000))))

print(q.qsize(), sys.getsizeof(q))

l = unpack_queue(q)

print(q.qsize(), sys.getsizeof(q), sys.getsizeof(l), q.qsize(), len(l))

1001 48
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
27

## A more complete example

- Logging before/after request
- Stats collection
  - Request duration
  - Number of retries
  - Errors received
- Rate limited/throttled requests
  - Ability to throttle ALL coroutines on demand (e.g. 503)
- Custom error handlers
- Custom request builders (build endpoint URL/request data from something like an endpoint ID)
- Be able to join multiple consumers/producers together with Queue in between
  - Or just have a single consumer working on a single queue and printing results

In [ ]:
import json

from collections import Counter

from dataclasses import field, dataclass
import sys
from aiohttp import ClientResponseError

class Sentinel: pass

async def id_response_unpacker(req_data, resp, queue, *args):
    await queue.put((0, (resp,)))
    await queue.put((0, Sentinel))

async def base_response_unpacker(req_data, resp, queue):
    for i in json.loads(resp):
        await queue.put((0, (*req_data, i)))
    await queue.put((0, Sentinel))

def id_request_builder(method, hostname, port, endpoint, i):
    return (method, f"http://{hostname}:{port}/{endpoint}/{i}")

def base_request_builder(method, hostname, port, endpoint):
    return (method, f"http://{hostname}:{port}/{endpoint}")

async def handle_error(e, q):
    print(f"HANDLING ERROR: {e}")
    if e.status == 429:
        q.retry()
    elif e.status == 503:
        q.lock(10)

@dataclass
class AsyncRequester:
    in_q: ThrottledQueue
    out_q: ThrottledQueue
    req_builder: object
    resp_unpacker: object
    error_handler: object
    log_prefix: str = "---"
    cntr: Counter = field(default_factory=Counter)

    async def consumer(self, idx):
        retrying = False
        while True:
            if not retrying:
                i, d = await self.in_q.get()
                print(self.log_prefix, d)
                if d == Sentinel:
                    await self.in_q.put((i, Sentinel))
                    print(self.log_prefix, f"worker {idx} exiting")
                    return
            async with request(*self.req_builder(*d)) as req:
                resp = await req.read()
                try:
                    req.raise_for_status()
                except ClientResponseError as e:
                    self.cntr["failure"] += 1
                    await error_handler(e, q)
                    retrying = True
                    # TODO implement retry limit
                    continue
                print(self.log_prefix, f"worker {idx} response for #{i}: {resp}")
                print(self.log_prefix, f"sending to queue: {resp}")
                await self.resp_unpacker(d, resp, self.out_q)
                self.cntr["success"] += 1

In [ ]:
t = asyncio.run(_fill_queue(ThrottledQueue(per_second=1), [("get", "0.0.0.0", "8080", "items")]))
t2 = ThrottledQueue(per_second=100)

a = AsyncRequester(
    in_q=t,
    out_q=t2,
    req_builder=base_request_builder,
    resp_unpacker=base_response_unpacker,
    error_handler=handle_error,
)

asyncio.run(
    asyncio.gather(
        asyncio.create_task(a.consumer(0))
    )
)
print("unpacked result:", asyncio.run(_unpack_queue(t2))[0:100])

--- ('get', '0.0.0.0', '8080', 'items')
--- worker 0 response for #0: b'[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 20

In [ ]:
t = asyncio.run(_fill_queue(ThrottledQueue(per_second=1), [("get", "0.0.0.0", "8080", "items")]))
t2 = ThrottledQueue(per_second=100)
results = asyncio.Queue()

all_customers_req = AsyncRequester(
    in_q          = t,
    out_q         = t2,
    req_builder   = base_request_builder,
    resp_unpacker = base_response_unpacker,
    error_handler = handle_error,
    log_prefix    = "+++",
)

customers_by_id_req = AsyncRequester(
    in_q          = t2,
    out_q         = results,
    req_builder   = id_request_builder,
    resp_unpacker = id_response_unpacker,
    error_handler = handle_error,
    log_prefix    = "___",
)

asyncio.run(
    asyncio.gather(
        asyncio.create_task(all_customers_req.consumer(0)),
        asyncio.create_task(customers_by_id_req.consumer(0)),
    )
)

+++ ('get', '0.0.0.0', '8080', 'items')
+++ worker 0 response for #0: b'[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 20

[None, None]

In [22]:
print(asyncio.run(_unpack_queue(results)))

KeyboardInterrupt: 